## Handwriting,signatures and initials detection in contract management __PART 2__

Dans l'article précédent, nous avons parlé de l'architecte de notre projet et de l'approche que nous avons suivie, et nous avons terminé l'étape d'annotation des contrats, nous avons utilisé localement le logiciel open source VOTT développé par Microsoft, pour annoter les contrats.

Dans cet article, nous continuerons à former notre modèle de détection d'objets en utilisant l'API de détection d'objets de TensorFlow. Étonnamment, c'est l'étape la plus simple par rapport à la collecte et à l'annotation des données.

Maintenant que nous avons fait tout ce qui précède, nous pouvons commencer à faire des trucs sympas. Ici, nous verrons comment vous pouvez former votre propre détecteur d'objets, et comme ce n'est pas aussi simple qu'il y paraît, nous allons voir:

* Comment organiser votre espace de travail / fichiers de formation
* Comment générer des enregistrements TF à partir de ces ensembles de données
* Comment configurer un pipeline de formation simple
* Comment former un modèle et suivre ses progrès
* Comment exporter le modèle résultant et l'utiliser pour détecter des objets.


# Préparer l'espace de travail

L'API de détection d'objets TensorFlow nécessite l'utilisation de la structure de répertoires spécifique fournie dans son référentiel GitHub. Il nécessite également plusieurs packages Python supplémentaires, des ajouts spécifiques aux variables PATH et PYTHONPATH et quelques commandes de configuration supplémentaires pour que tout soit configuré pour exécuter ou entraîner un modèle de détection d'objet.

suivez attentivement les instructions, car une mauvaise configuration peut provoquer des erreurs difficiles à suivre.

En partant de l'hypothèse que vous venez de créer votre nouvel environnement virtuel (que ce soit tensorflow_cpu , tensorflow_gpu ou tout autre nom que vous auriez pu utiliser), certains packages doivent être installés avant d'installer les 
modèles.

- pip install pillow
- pip install lxml
- pip install Cython
- pip install contextlib2
- pip install jupyter
- pip install matplotlib
- pip install pandas
- pip install opencv-python

 #### Téléchargement des modèles TensorFlow 
* Créez un nouveau dossier sous le chemin de votre choix et nommez-le par exemple my-project.
* télécharger les modèles https://github.com/tensorflow/models, vous pouvez soit utiliser Git pour cloner la version TensorFlow Models  dans le my-project dossier, soit le télécharger simplement sous forme de ZIP et extraire son contenu dans le my-project dossier. 

**Remarque**: Le code du référentiel de modèles TensorFlow (qui contient l'API de détection d'objets) est continuellement mis à jour par les développeurs. Parfois, ils apportent des modifications qui rompent les fonctionnalités avec les anciennes versions de TensorFlow. Il est toujours préférable d'utiliser la dernière version de TensorFlow et de télécharger le dernier référentiel de modèles.

### COCO API installation

Téléchargez le cocoapi et copiez le sous-dossier pycocotools dans le répertoire /models/research si vous êtes intéressé à utiliser les métriques d'évaluation COCO. Les métriques par défaut sont basées sur celles utilisées dans l'évaluation Pascal VOC. Pour utiliser les métriques de détection d'objets COCO, ajoutez metrics_set: "coco_detection_metrics"au eval_configmessage dans le fichier de configuration

In [ ]:
git clone https://github.com/cocodataset/cocoapi.git
cd cocoapi/PythonAPI

cp -r pycocotools <path_to_my-project/models/research/

Alternativement, Vous pouvez  installer pycocotools en utilisant pip :

### Compilation de Protobuf 


Ensuite, compilez les fichiers Protobuf, qui sont utilisés par TensorFlow pour configurer le modèle et les paramètres de formation. Avant que le framework puisse être utilisé, les bibliothèques Protobuf doivent être téléchargées et compilées.

Cela devrait être fait comme suit:

* vous pouvez l'installer directement depuis pip : **pip install pycocotools**

Ensuite  dans un nouveau terminal, dans le my-project/models/research/  répertoire et exécutez la commande suivante:

protoc  object_detection/protos/*.proto --python_out=.

Si vous obtenez des erreurs lors de la compilation, vous utilisez peut-être un compilateur protobuf incompatible. Si tel est le cas, utilisez l'installation manuelle suivante

    * Rendez-vous sur la page des versions du protocole : https://github.com/protocolbuffers/protobuf/releases
    * Téléchargez la dernière protoc-*-*.zipversion (par exemple protoc-3.11.4-win64.zip  pour Windows 64 bits,protoc-3.11.4-linux-x86_32.zip)
    * Extraire le contenu du téléchargé protoc-*-*.zip dans un répertoire <PATH_TO_PB> de votre choix 

    * Ajoutez <PATH_TO_PB>à votre Path variable d'environnement
    
    


Si vous etes sur windows Malheureusement, cette comande  ne fonctionne pas . Chaque fichier .proto du répertoire \ object_detection \ protos doit être appelé individuellement par la commande.

## Ajouter les variables d'environnement nécessaires 

* **1.** Installez le my-project\models\research\object_detection package en exécutant ce qui suit à partir de my-project\models\research:


* **2.** Une variable PYTHONPATH doit être créée qui pointe vers les répertoires / models/research/slim. Pour ce faire, exécutez les commandes suivantes (à partir de n'importe quel répertoire):

Créez maintenant un nouveau dossier sous my-project et appelez-le workspace. C'est au sein de la workspaceque nous conserverons l'ensemble de nos formations. Passons maintenant sous l'espace de travail et créons un autre dossier nommé training_Faster_RCNN . Maintenant, notre structure de répertoire devrait être la suivante:

Le training_Faster_RCNN dossier sera notre dossier de formation , qui contiendra tous les fichiers liés à notre modèle de formation. Il est conseillé de créer un dossier de formation distinct chaque fois que nous souhaitons former un modèle différent. La structure typique des dossiers de formation est illustrée ci-dessous.

Voici une explication pour chacun des dossiers / fichiers présentés dans l'arborescence ci-dessus:

* **annotations**: Ce dossier sera utilisé pour stocker tous les *.csv fichiers et les *.record fichiers TensorFlow respectifs , qui contiennent la liste des annotations pour nos images d'ensemble de données.

* **images**: Ce dossier contient une copie de toutes les images de notre jeu de données, ainsi que les *.xml fichiers respectifs

* **images\train**: Ce dossier contient une copie de toutes les images, et les *.xml fichiers respectifs , qui seront utilisés pour former notre modèle.
* **images\test**: Ce dossier contient une copie de toutes les images, et les *.xml fichiers respectifs , qui seront utilisés pour tester notre modèle.

* **pre-trained-model**: Ce dossier contiendra le modèle pré-formé de notre choix, qui sera utilisé comme point de départ pour notre travail de formation.

* **training**: Ce dossier contiendra le fichier de configuration du pipeline de formation *.config, ainsi qu'un *.pbtxt fichier de mappage d'étiquettes et tous les fichiers générés lors de la formation de notre modèle.

Si vous ne comprenez pas la plupart des choses mentionnées ci-dessus, ne vous inquiétez pas, car nous verrons comment tous les fichiers sont générés plus loin.

## Annoter des images

C'est deja fait dans l'article précedent .

Une fois que vous avez fini d'annoter votre jeu de données d'image, il est de règle générale de n'en utiliser qu'une partie pour la formation, et le reste est utilisé à des fins d'évaluation .

En règle générale, le ratio est de 80% / 20%, c'est-à-dire que 80% des images sont utilisées pour la formation et les 20% restants sont maintenus pour les tests, mais vous pouvez choisir le ratio qui convient à vos besoins.

Une fois que vous avez décidé de la façon dont vous allez fractionner votre jeu de données, copiez toutes les images d'entraînement, ainsi que leurs *.xml fichiers correspondants , et placez-les dans le training_Faster_RCNN\images\train dossier. De même, copiez toutes les images de test, avec leurs *.xml fichiers, et collez-les à l'intérieur training_Faster_RCNN\images\test.

j'ai mis un script simple nommé partition_dataset.py  qui automatise le processus ci-dessus

Pour utiliser le script, Ensuite, en supposant que vous avez toutes vos images et *.xml fichiers à l'intérieur training_Faster_RCNN\images, exécutez simplement la commande suivante:

Une fois le script terminé, il devrait exister deux nouveaux dossiers sous training_Faster_RCNN\images, à savoir training_Faster_RCNN\images\train et training_Faster_RCNN\images\test contenant respectivement 80% et 20% des images (et *.xml fichiers). Pour éviter la perte de fichiers, le script ne supprimera pas les images sous training_Faster_RCNN\images. Une fois que vous avez vérifié que vos images ont été copiées en toute sécurité, vous pouvez supprimer les images sous training_Faster_RCNN/images manuellement

## Définissez votre carte d'étiquettes

La mappe d'étiquettes indique que chaque classe est mappée sur un identifiant. Utilisez un éditeur de texte pour créer un nouveau fichier et enregistrez-le sous label_map.pbtxt dans le dossier   training_Faster_RCNN\training. (Assurez-vous que le type de fichier est .pbtxt, pas .txt!) Dans l'éditeur de texte, copiez ou saisissez la carte d'étiquettes au format ci-dessous (l'exemple ci-dessous est la carte d'étiquettes pour mon détecteur de signature ....):

## génération de TFRecords
vous devez vérifier si le format de votre jeu de données correspond au format requis par tensorflow. S'il ne correspond pas, vous devez adapter votre jeu de données.


L'API de détection d'objets TensorFlow ne prend pas les fichiers json ou xml  en entrée, mais elle a besoin des fichiers TFRecord format pour former le modèle. 

Pour ce faire, cela depend de format que vous avez exportés vos actifs dans VOTT :


*  **VoTT (generic JSON schema)** :  il faut  utiliser le script  make_official_dataset.py Le make_official_dataset.py importe ..training_Faster_RCNN/images/detection-export.json où toutes les annotations de bbox sont stockées, et crée un ensemble de train et de test  TFRrecords. Le script créera également un images/tfrecords répertoire où le train et l'ensemble de test sont enregistrés au .tfrecords format. 

**python3 make_official_dataset.py --labels "signature" "Handwrit" "intial"  --img_dir training_Faster_RCNN/images** .


* **format Pascal VOC**: il y a deux étapes:

    *Conversion des *.xml fichiers individuels en un *.csv fichier unifié pour chaque ensemble de données.
    
    *Conversion des *.csv fichiers de chaque jeu de données en *.record fichiers (format TFRecord).
    
    
     1.  Conversion *.xmlen *.csv
    Pour ce faire , vous pouvons exécuter le script **xml_to_csv.py** dans mon repot git  qui effectue une itération dans tous les *.xml fichiers dans ..\images\train et ..\images\test dossiers, et génère un *.csv pour chacun des deux.

Une fois que ce qui précède est fait, il devrait y avoir 2 nouveaux fichiers sous le training_Faster_RCNN\annotations dossier, nommés test_labels.csv et train_labels.csv, respectivement.

2. Conversion de *.csv à *.record
        Ouvrez le fichier generate_tfrecord.py qui se trouve dans mon repot git dans un éditeur de texte. Remplacez la carte d'étiquettes à partir de la ligne 31 par votre propre carte d'étiquettes, où chaque objet se voit attribuer un numéro d'identification. Cette même attribution de numéro sera utilisée lors de la configuration du fichier label_map.pbtxt

Par exemple, dans notre cas. Vous allez remplacer le code suivant dans generate_tfrecord.py:

In [7]:
# TO-DO replace this with label map
def class_text_to_int(row_label):
    if row_label == 'signature':
        return 1
    elif row_label == 'Handwrit':
        return 2
    elif row_label == 'intial':
        return 3
    else:
        None

Une fois que ce qui précède est fait, il devrait y avoir 2 nouveaux fichiers sous le training_Faster_RCNN\annotations dossier, nommés test.recordet train.record, respectivement.

# Téléchargez le modèle Faster-RCNN-Inception-V2-COCO depuis le zoo modèle de TensorFlow

https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md

Ici vous trouverez une liste des modèles téléchargeables. Dans ce aarticle, nous utilisons le modèle plus rapide_rcnn_inception_v2_coco.

TensorFlow fournit plusieurs modèles de détection d'objets (classificateurs pré-formés avec des architectures de réseaux neuronaux spécifiques) dans son zoo modèle . Certains modèles (tels que le modèle SSD-MobileNet) ont une architecture qui permet une détection plus rapide mais avec moins de précision, tandis que certains modèles (tels que le modèle Faster-RCNN) offrent une détection plus lente mais avec plus de précision.


vous pouvez Téléchargez le modèle ici
http://download.tensorflow.org/models/object_detection/faster_rcnn_inception_v2_coco_2018_01_28.tar.gz
    
ou via la line de comande 
#From training_Faster_faster_rcnn\pre-trained-model
- wget http://download.tensorflow.org/models/object_detection/faster_rcnn_inception_v2_coco_2018_01_28.tar.gz
- tar -zxvf faster_rcnn_inception_v2_coco_2018_01_28.tar.gz
- rm -r faster_rcnn_inception_v2_coco_2018_01_28.tar.gz

## Personnalisez un fichier de configuration pour votre modèle

Dans ce fichier, vous pouvez configurer les paramètres et votre entraînement. nous l'utiliserons faster_rcnn_inception_v2.configet le personnaliserons en fonction de nos besoins. Si vous souhaitez utiliser un autre modèle, reportez-vous au référentiel Github officiel de tensorflow: https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md


Enfin, le pipeline d'apprentissage de détection d'objets doit être configuré. Il définit quel modèle et quels paramètres seront utilisés pour l'entraînement de notre modèle . C'est la dernière étape avant de commencer l'entraînement!

Accédez à <PATH_TO_MODELS_FOLDER>/models/research/object_detection/samples/configs et copiez le fichier fast_rcnn_inception_v2_pets.config dans le répertoire <PATH_TO_MY-PROJECT_FOLDER>/my-project/workspace/training_faster_rcnn/training . Ensuite, ouvrez le fichier avec un éditeur de texte. Il y a plusieurs modifications à apporter au fichier .config, principalement le nombre de classes et d'exemples et l'ajout des chemins d'accès aux données d'entraînement.

Apportez les modifications suivantes au fichier fast_rcnn_inception_v2_pets.config.

##### Changer le nombre de classes:Ligne 9

### Ajoutez le chemin d'accès au modèle faster_rcnn_inception_v2_coco_2018_01_28 téléchargé .

#### Remplacez num_examples par le nombre d'images que vous avez dans le répertoire \images\test.

In [ ]:
#Ligne 130
eval_config: {
  metrics_set: "coco_detection_metrics"
  num_examples: 78 #here
}


### Ajoutez le chemin absolu à votre ensemble train et teste.

In [ ]:
#Lignes 123 et 125
train_input_reader {
  ##### change path
  label_map_path:"PATH_TO_BE_CONFIGURED/label_map.pbtxt"
  tf_record_input_reader {
  ##### change path
    input_path:"PATH_TO_BE_CONFIGURED/train.tfrecords"
  }

In [ ]:
##Lignes 135 et 137
eval_input_reader {
  ##### change path here ... (abs path)
  label_map_path:"PATH_TO_BE_CONFIGURED/label_map.pbtxt"
  shuffle: false
  num_readers: 1
  tf_record_input_reader {
  ##### change path here ... (abs path)
    input_path:"PATH_TO_BE_CONFIGURED/tset.tfrecords"
  }

Enregistrez le fichier une fois les modifications apportées. C'est ça! Le travail de formation est tout configuré et prêt à démarrer!

## Former le modèle


Avant de commencer à former notre modèle, allons copier le my-project/models/research/object_detection/model_main.py script et collez-le directement dans notre training_faster_rcnn dossier. Nous aurons besoin de ce script pour former notre modèle.

Maintenant, pour lancer un nouveau travail de formation, cd dans le training_faster_rcnn dossier et tapez ce qui suit:

In [ ]:
python model_main.py --alsologtostderr --model_dir=training/ --pipeline_config_path=training/faster_rcnn_inception_v2_pets.config

## Surveiller la progression de formation à l'aide de TensorBoard
Vous pouvez afficher la progression du travail de formation à l'aide de TensorBoard. Pour ce faire, ouvrez une nouvelle instance , activez votre environnement virtuel, accédez au répertoire training_faster_rcnn et exécutez la commande suivante:


In [ ]:
tensorboard --logdir=training\

Cela va créer une page Web sur votre ordinateur local à YourPCName: 6006, qui peut être consultée via un navigateur Web. La page TensorBoard fournit des informations et des graphiques qui montrent comment la formation progresse. Un graphique important est le graphique de perte, qui montre la perte globale du classificateur au fil du temps.

<img src="data2/loss.png" width="600"/>

# Exportation d'un graphique d'inférence formé  Exporter le graphique d'inférence


Maintenant que la formation est terminée, la dernière étape consiste à générer le graphe d'inférence(fichier .pb),qui sera ensuite utilisé pour effectuer la détection d'objet. Dans le dossier  training_faster_rcnn, exécutez la commande suivante, où «XXXX» avec le numéro le plus élevé  dans «model.ckpt-XXXX» :

Copiez le /models/research/object_detection/export_inference_graph.pyscript et collez-le directement dans votre training_faster_rcnn.


In [ ]:
python export_inference_graph.py --input_type image_tensor --pipeline_config_path training/faster_rcnn_inception_v2_pets.config --trained_checkpoint_prefix training/model.ckpt-XXXX --output_directory trained-inference-graphs/output_inference_graph_v1.pb

## RÉSULTATS

Ce projet comportait deux éléments principaux: 
    * La détection d'objets d'écriture manuscrite, signatures , intials 
    *  l'OCR d'écriture manuscrite (La transcription du texte manuscrit).

Dans cette partie, nous nous concentrons sur les mesures de performances pour le modèle de détection d'objets.

Les mesures de performance ont été calculées comme décrit ci-dessous:

Notre ensemble de tests comprenait 80 pages de contrats provenant de la même source de données de contrats gouvernementaux, mais pas encore vus par notre modèle. 

Si vous avez rencontré un projet impliquant la détection d'objets, vous connaissez peut-être la métrique de choix dans la communauté: mean Average Precision(mAP).

Commençons par d calculer l'intersection sur l'union (IoU),pour notre tâche, car nous étions intéressés par notre capacité à récupérer uniquement les zones d'intérêt dans un texte imprimé par le modèle.

#### IoU
L'intersection sur l'Union est un rapport entre l'intersection et l'union des boîtes prédites et des boîtes de vérité. Cette statistique est également connue sous le nom d'index Jaccard et a été publiée pour la première fois par Paul Jaccard au début des années 1900.


<img src="data2/iuo.PNG" width="300"/>



Nous utilisons ensuite ce rapport pour déterminer si une boîte englobante prédite est:

* Vrai positif (TP): IoU> 0,5 et classe correcte détectée;
* Faux positif (FP): IoU <0,5 ou boîte englobante dupliquée;
* Faux négatif (FN): IoU> 0,5 mais mauvaise classe détectée.
    
Une fois nos métriques calculées (TP, FP, TN), nous pouvons calculer deux autres métriques cruciales, la précision et le rappel

<img src="data2/metric.PNG" width="300"/>

et tracer la courbe Précision-Rappel (PR). La précision moyenne est alors l'aire sous la courbe PR interpolée . Enfin, le mAP est la moyenne des AP calculés pour toutes les classes .

Pour une compréhension plus complète de ce qui précède, ou si vous n'êtes pas familier avec la notion de mAP, je vous suggère de lire ce blog .

Dans ce quide , nous avons expliqué comment nous utilisons l' API COCO pour calculer les métriques d'évaluation.

Globalement, nous atteignons une performance de mAP = 0,87 pour un seuil de IoU = 0,5 qui est la norme.

Lors de la définition du seuil IoU à 0,75 (ce qui signifie qu'une prédiction n'est vraiment positive que si IoU> 0,75), nous avons observé une baisse très significative des performances à mAP = 0,39. Cependant, dans la pratique, un seuil IoU de 0,5 est plus que suffisant car notre objectif principal est de localiser où se trouve l'object dans le document, plus que d'avoir un cadre de délimitation précis.

<img src="data2/out-put.jpg" width="600"/>

# CONCLUSION

Dans ce post, nous nous sommes concentrés sur la détection de signature car c'est un problème utile que nous avons rencontré lors de nombreux projets clients. Cependant, ce travail peut être étendu à tout autre type d'écriture ou «éléments» qui pourraient être mal interprétés par un OCR.

En effet, l'extraction d'entités et d'autres tâches reste difficile en raison des nombreuses variations que l'on peut trouver dans une population de documents: mauvaise qualité et contraste, salissures / documents froissés, polices, caractères non latins etc ... La présence de tableaux et d'images peut aussi affecter la précision de l'OCR. Ces erreurs introduisent des lettres sautées et mal lues et rendent l'analyse et l'extraction d'entités plus difficiles.